In [1]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import itertools

In [2]:
original = cv.VideoCapture('./1tagvideo.mp4')
if (original.isOpened() == False):
	print("Error opening the video file")
else:
  # Get fps
  fps = int(original.get(5))
  print("Frame Rate is ",fps,"frames per second")	
  # Get total frame count
  frame_count = original.get(7)
  print("Frame count : ", frame_count)



Frame Rate is  26 frames per second
Frame count :  787.0


In [3]:
#Fit rectangles given the list of corners
def detect_rectangles(corners:list()):
    point_pairs = []
    # Create a list of point pairs
    for point_pairs in itertools.combinations(corners,2):
        p1,p2= point_pairs[0],point_pairs[1]
        dist = np.linalg.norm(p1-p2)
        if(dist>50):
            slope = 0
            center =  np.array([(p1[0]+p2[0])/2,(p1[1]+p2[1])/2])
            point_pairs.append([p1,p2,slope,center,dist])
               
    possible_corners = np.array(point_pairs,dtype=object)
    possible_corners = possible_corners[np.argsort(possible_corners[:,-1])][::-1]
    
    rectangles = []
    center_lengths = []
    
    # Create a list of points such that the lines formed by them have similar lengths and mid point coordinates
    for line_pairs in itertools.combinations(possible_corners,2):
        n,m = line_pairs[0],line_pairs[1]
        if(np.abs(n[-1]-m[-1])>10):
            continue
        if (np.linalg.norm(m[3]-n[3])>10):
            continue
        rectangles.append(np.array([tuple(n[0].tolist()),tuple(m[0].tolist()),tuple(n[1].tolist()),tuple(m[1].tolist())]))
        center_lengths.append(np.array([n[3],np.linalg.norm(n[0]-m[0])],dtype = object))
    return rectangles,center_lengths

In [4]:
def plot_polygon(img, lov):
    for i in lov:
        pts = np.array(i, np.int32)
        pts = pts.reshape((-1,1,2))
        img_rect = cv.polylines(img,[pts],True,(0,255,255))
    return img_rect

In [5]:
# Calculate Homography
def calculate_homography(world_corners,image_corner):
    #4 points in the world frame
    Xw = world_corners
    # 4 points in the camera frame
    xc = image_corner
    # intermediary matrix to calculate the homography
    matrix_A = np.matrix(np.zeros((8,9)))
    # Populate the martrix
    for a,b in zip(enumerate(Xw),enumerate(xc)):
        i,j,n,m = a[0],b[0],a[1],b[1]
        matrix_A[i+j,:] = -n[0],-n[1],-1,0,0,0,m[0]*n[0],m[0]*n[1],m[0]
        matrix_A[(i+j)+1,:] = 0,0,0,-n[0],-n[1],-1,m[1]*n[0],m[1]*n[1],m[1]
    # Calculate the psudo inverse of the matrix
    U,S,V = np.linalg.svd(matrix_A)
    # The vector with the smallest eigen value is the solution to the equation
    H = np.reshape(V[-1],[3,3])
    return H/H[-1,-1]

In [6]:
#Decode the tag given a cropped image 
def decode_tag(tag):
    # Threshold the image to a binary image
    tag = cv.cvtColor(tag,cv.COLOR_BGR2GRAY)
    _,tag = cv.threshold(tag,127,255,cv.THRESH_OTSU+cv.THRESH_OTSU)
    # Split the image into 8 equal segments
    tag_grids = np.array_split(tag,8)
    msg = np.zeros((8,8))
    # Iterate over all segments and average each segment into the binary value
    for i,tag_grid in enumerate(tag_grids):
        for j,grid in  enumerate(np.array_split(tag_grid,8,axis=1)):
            if np.count_nonzero(grid) < 0.5*grid.size:
                msg[i][j] = 0
            else:
                msg[i][j] =1
    # assign the orientation of the tag based on the 3rd row/column
    if msg[2][2]:
        ori = 180
    elif msg[2][5]:
        ori = 90
    elif msg[5][2]:
        ori = 270
    elif msg[5][5]:
        ori = 0
    else :
        ori = None
    # Assign the ID of the tag
    idx = (int(msg[3,3])*1+int(msg[3,4])*2+int(msg[4,4])*4+int(msg[4,3])*8)
    # Rotate the ID given the orientation
    idx = ((idx<<(ori//90)&0b1111)|(idx>>(4-(ori//90))))
    # If the border condition is not met then return None
    if((msg[0:2]!=0).any() and (msg[:][0:2]!=0).any() and (msg[6:-1][:]!=0).any() and (msg[:][6:-1]!=0).any()):
        return np.nan,None,None
    
    return msg,ori,idx

In [7]:
# Warp the image with the given size and template image
def Warping(src, homography, size, template = None):
    y, x = np.indices((size[0], size[1]))
    ## Create an array with values equal to coordinates of the point
    cam_pts = np.stack((x.ravel(), y.ravel(), np.ones(x.size)))
    h_inv = np.linalg.inv(homography)
    
    ## Find the transformation that maps the camera point to the world point
    cam_pts = h_inv.dot(cam_pts)
    ## Normalize so that the Z is 1
    cam_pts /= cam_pts[2,:]

    ## Floor the float values to a interger value
    xw, yw = cam_pts[:2,:].astype(int)
    # padding
    xw[xw >=  src.shape[1]] = src.shape[1]
    xw[xw < 0] = 0
    yw[yw >=  src.shape[0]] = src.shape[0]
    yw[yw < 0] = 0
    ## If a template image is provided then map that to world frame
    if (type(template)==np.ndarray):
        src[yw, xw, :] = template[y.ravel(), x.ravel(), :]
        return src
    ## Map the world frame to the given the camera frame
    else:
        warped_image = np.zeros((size[0],size[1], 3))
        warped_image[y.ravel(), x.ravel(), :]= src[yw, xw, :]
        return warped_image

In [8]:
# Sort the points so that the are in anti clockwise 
def sort_points(points):
    left_most = points[np.argmin(points[:,0])]
    right_most = points[np.argmax(points[:,0])]
    topmost = points[np.argmin(points[:,1])]
    bottommost = points[np.argmax(points[:,1])]
    return [left_most,bottommost,right_most,topmost]
    pass
    

In [11]:
def process_video():
    frames = []
    out = cv.VideoWriter('outpy.avi',cv.VideoWriter_fourcc('M','J','P','G'), 26, (1920,1080))
    testudo  =cv.imread('testudo.jpg')
    previous_tag=None
    while( original.isOpened()):
        #  original.read() methods returns a tuple, first element is a bool 
        # and the second is frame
        ret, frame =  original.read()
        if ret == True:
        #Split the channels and invert color of the frame as we are only interested in the red channel
            frames.append(frame)
            gaussian = np.array([[-1,-2,-1], [-2 ,13.5,-2], [-1,-2,-1]])# Sharpen the image to hightlight edges
            img = frame #assign the frame of interest
            new_img = cv.medianBlur(img, 5) # Blur the image slightly to remove noise
            new_img = cv.filter2D(new_img,-1,gaussian) #Sharpen the image to get the edges more promanent 
            gray = cv.cvtColor(new_img,cv.COLOR_BGR2GRAY) #convert to grayscale

            #Perform corner detection
            corners = cv.goodFeaturesToTrack(gray,30,0.1,35) # only 30 points are selected
            corners = np.int0(corners)
            list_of_corners = []
            for i in corners:
                x,y = i.ravel()
                list_of_corners.append(np.array([x,y]))
            # Given the list of corners, fit quads to the images
            rectangles,d=detect_rectangles(list_of_corners)
            tags = []
            # For every quad calculate the Homography matrix that converts the world point to the camera plane
            for i, rectangle in enumerate(rectangles):
                # MAKE IT TO THE CLOSEST MULTIPLE OF 8 the side length of tag
                max_frame_size = int(d[i][1])+(int(d[i][1])%8)
                # The points on the image plane that will be mapped to the world plane
                PoF = [[0,0],[max_frame_size,0],[max_frame_size,max_frame_size],[0,max_frame_size]]
                # Sort the list of points such that the points are in the clockwise sense
                rectangle = sort_points(rectangle)
                # Calculate the homography using SVD
                homo = calculate_homography(rectangle[::-1],PoF)
                try:
                    warped_img = np.uint8(Warping(img,homo,(max_frame_size,max_frame_size)))
                    # Decode the same image assuming its a TAG
                    msg,ori,idx = decode_tag(warped_img)
                except:
                    continue
                # Add it to the list of tags
                tags.append([rectangle,msg,ori,idx,max_frame_size,homo])
                
                # For every quad that returned a valid tag info superimpose a image
                tag = tags[-1]
                if(np.isnan(tag[1]).any() or type(tag[2])==type(None) or type(tag[3])==type(None)):
                    continue
                if(tag[3]!=7):
                    continue
                previous_tag = tag
                
        else:
            break
        print(len(frames)) 
        


In [12]:
if __name__=="__main__":
    process_video()

AttributeError: 'tuple' object has no attribute 'append'